In [1]:
import findspark
import os
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
# SPARK_HOME='/home/qiany/.conda/envs/py37'
# os.environ['SPARK_HOME'] = '/home/qiany/.conda/envs/py37'
findspark.init(SPARK_HOME)

In [2]:
import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
#from hdfs import InsecureClient
import numpy as np
#from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
#import igraph
#from igraph import Graph
import geofeather
from pyspark.storagelevel import StorageLevel

In [3]:
tin_file = input("Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file:")
print("\n********************")

# get the directory to the TIN file
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or SigSpatial(1):")
if directory_type == '0':
    directory = 'hdfs_data'
else:
    directory = 'SigSpatial_data'
    
# get the basename to the TIN file
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

# get the filename of the TIN file
tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

# get the type of TIN file: off, tri, etc
tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)
print("\n********************")

Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file: cos_sum.tri



********************
tin_directory:  


Is the data stored in hdfs(0) or SigSpatial(1): 0


tin_basename:  cos_sum.tri
tin_filename:  cos_sum
tin_extension:  .tri

********************


In [4]:
# filtra is the order of each vertex, the order is obtained by ranking elevation values of vertices
filtra = input("Do you have filtration data?") or "yes"
# filtra = 'yes'

if filtra.lower() == 'no':    
    Basic_Data = input("Do you have basic pts and tri data?")
    
Num_executor = input("spark.executor.instances:") or "4"
Num_core_per_executor = input("spark.executor.cores:") or "5"
Memory_executor = input("spark.executor.memory? Please end with 'g':") or "10g"
MemoryOverhead_executor = input("spark.executor.memoryOverhead? Please end with 'g':") or "2g"

# Num_core_per_driver = Num_core_per_executor
# Memory_driver = Memory_executor
# MemoryOverhead_driver = MemoryOverhead_executor

Num_core_per_driver = '5'
Memory_driver = '12g'
MemoryOverhead_driver = '4g'

Num_shuffle_partitions = input("spark.sql.shuffle.partitions:") or "20"

Do you have filtration data? 
spark.executor.instances: 
spark.executor.cores: 
spark.executor.memory? Please end with 'g': 
spark.executor.memoryOverhead? Please end with 'g': 
spark.sql.shuffle.partitions: 20


In [5]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [6]:
os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [7]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

# set the Spark app name
spark_app_name = "TopoSimp_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '200') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.getOrCreate()

spark_app_name: TopoSimp_cos_sum_03072024_1255


In [8]:
import sys
sys.path.append("/local/data/yuehui/pyspark/FormanGradient/code")

In [9]:
import graphframes
from graphframes import GraphFrame
from graphframes import *
from graphframes.lib import Pregel

In [10]:
# read_ver_order() is a function used to read vertices from a csv file
def read_ver_order(filtra, directory, tin_filename):
    '''
    this function has three input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a TIN file
    tin_filename: a string denoting the file name of a TIN without extension, e.g., Canyon_Lake_Gorge_TX
    '''
    if filtra.lower() == 'yes': # 'yes' means we have already computed the filtra value for each vertex
        schema_ver_origin = StructType([ \
            StructField("x",FloatType(),True), \
            StructField("y",FloatType(),True), \
            StructField("ele",FloatType(),True), \
            StructField("self_index",IntegerType(),True), \
            StructField("self_order",IntegerType(),True) \
          ])
        
        hdfs_tin_pts_origin = directory + "/" + tin_filename + '_filtra_pts_origin.csv'
    
        df_ver_order = spark.read.format("csv") \
              .option("header", False) \
              .schema(schema_ver_origin)\
              .load(hdfs_tin_pts_origin)
        print("Number of partitions for df_ver_order:", df_ver_order.rdd.getNumPartitions())
        
    if filtra.lower() == 'no': # 'no' means we need to rank vertices to get their filtra values
        schema_ver_origin = StructType([ \
            StructField("x",FloatType(),True), \
            StructField("y",FloatType(),True), \
            StructField("ele",FloatType(),True), \
            StructField("self_index",IntegerType(),True) \
          ])
        
        hdfs_tin_pts_origin = directory + "/" + tin_filename + '_pts_origin.csv'
    
        df_ver_origin = spark.read.format("csv") \
              .option("header", False) \
              .schema(schema_ver_origin)\
              .load(hdfs_tin_pts_origin)
        
        print("Number of partitions for df_ver_origin:", df_ver_origin.rdd.getNumPartitions())
        
        # define a window for the ordering
        # row_number() function along with partitionBy() of other column populates the row number by group
        # Since we want to order the whole DataFrame, so we don't need the partitionBy() function
        w = Window().orderBy(col('ele').asc())
        df_ver_order = df_ver_origin.withColumn("self_order", F.row_number().over(w) -1) # let the row number start from 0
        print("Number of partitions for df_ver_order:", df_ver_order.rdd.getNumPartitions())
        
    return df_ver_order

In [11]:
# read vertices
df_ver_order = read_ver_order(filtra, directory, tin_filename)
df_ver_order.printSchema()

Number of partitions for df_ver_order: 1
root
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- ele: float (nullable = true)
 |-- self_index: integer (nullable = true)
 |-- self_order: integer (nullable = true)



In [12]:
df_ver_order.show()

+--------+---+---------+----------+----------+
|       x|  y|      ele|self_index|self_order|
+--------+---+---------+----------+----------+
|     0.0|0.0|      2.0|         0|       621|
|0.392699|0.0|  1.92388|         1|       609|
|0.785398|0.0|  1.70711|         2|       588|
|  1.1781|0.0|  1.38268|         3|       549|
|  1.5708|0.0| 0.999996|         4|       501|
|  1.9635|0.0| 0.617312|         5|       444|
| 2.35619|0.0| 0.292896|         6|       386|
| 2.74889|0.0|0.0761219|         7|       356|
| 3.14159|0.0|      0.0|         8|       277|
| 3.53429|0.0|0.0761198|         9|       348|
| 3.92699|0.0| 0.292893|        10|       378|
| 4.31969|0.0| 0.617317|        11|       448|
| 4.71239|0.0|      1.0|        12|       505|
| 5.10509|0.0|  1.38269|        13|       553|
| 5.49779|0.0|  1.70711|        14|       589|
| 5.89049|0.0|  1.92388|        15|       610|
| 6.28319|0.0|      2.0|        16|       622|
| 6.67588|0.0|  1.92388|        17|       611|
| 7.06858|0.0

In [12]:
# read_tri_order() is a function used to read triangles from a csv file
def read_tri_order(filtra, directory, tin_filename):    
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a TIN file
    tin_filename: a string denoting the file name of a tin without extension, e.g., 827_monviso
    '''
    if filtra.lower() == 'yes':
        hdfs_tin_tri_origin = directory + "/" + tin_filename + '_filtra_tri_origin.csv'
    else: # filtra.lower() == 'no'
        hdfs_tin_tri_origin = directory + "/" + tin_filename + '_tri_origin.csv'
        
    schema_tri_origin = StructType([ \
        StructField("v1",IntegerType(),True), \
        StructField("v2",IntegerType(),True), \
        StructField("v3",IntegerType(),True), \
        StructField("tri_order",IntegerType(),True) \
      ])

    df_tri_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tri_origin)\
          .load(hdfs_tin_tri_origin)
    print("Number of partitions for df_tri_origin:", df_tri_origin.rdd.getNumPartitions())
        
    return df_tri_origin

In [13]:
# read triangles
df_tri_origin = read_tri_order(filtra, directory, tin_filename)
df_tri_origin.printSchema()

Number of partitions for df_tri_origin: 1
root
 |-- v1: integer (nullable = true)
 |-- v2: integer (nullable = true)
 |-- v3: integer (nullable = true)
 |-- tri_order: integer (nullable = true)



In [14]:
# replace_ver() replaces the original index of each vertex with filtra value in df_tri_origin
def replace_ver(df_ver_order, df_tri_origin):
    '''
    df_ver_order: a DataFrame storing sorted vertices with filtra values
    df_tri_origin: a DataFrame storing triangles from a TIN
    '''
    df_tri_order_v1 = df_tri_origin.join(df_ver_order, df_tri_origin.v1 == df_ver_order.self_index, "inner")
    df_tri_order_v1 = df_tri_order_v1.select(col("tri_order"), col("v1"), col("v2"), col("v3"),col("self_order").alias("r1"), col("ele").alias("r1_ele"))

    df_tri_order_v2 = df_tri_order_v1.join(df_ver_order, df_tri_order_v1.v2 == df_ver_order.self_index, "inner")
    df_tri_order_v2 = df_tri_order_v2.select(col("tri_order"), col("v1"), col("v2"), col("v3"), col("r1"), col("self_order").alias("r2"), col("r1_ele"), col("ele").alias("r2_ele"))

    df_tri_order_v3 = df_tri_order_v2.join(df_ver_order, df_tri_order_v2.v3 == df_ver_order.self_index, "inner")
    df_tri_order_v3 = df_tri_order_v3.select(col("tri_order"), col("v1"), col("v2"), col("v3"), col("r1"), col("r2"), col("self_order").alias("r3"), col("r1_ele"), col("r2_ele"), col("ele").alias("r3_ele"))

    df_tri_order = df_tri_order_v3.select(col("tri_order"), col("r1"), col("r2"), col("r3"), col("r1_ele"), col("r2_ele"), col("r3_ele"))

    return df_tri_order

In [15]:
# replaces original vertex index with filtra values
df_tri_order = replace_ver(df_ver_order, df_tri_origin)
df_tri_order.printSchema()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)



In [16]:
# sort the extreme vertices of a triangle in ascending order, e.g., (2,5,3) -> (2,3,5)

'''
def get_tri_array(r1, r2, r3):
# get_multi_pt_index is used to obtain the adjacent vertexes index, including the vertex itself
# pt_list1: partial adjacent vertex indexes of join result 1
# pt_list2: partial adjacent vertex indexes of join result 2
    tri = [r1, r2, r3]    
    tri.sort(reverse=True)
    
    return tri

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_tri_array_udf = udf(get_tri_array, ArrayType(IntegerType()))
df_tri_order = df_tri_order.withColumn("tri", get_tri_array_udf(df_tri_order.r1, df_tri_order.r2, df_tri_order.r3))
'''

df_tri_order = df_tri_order.withColumn("tri_origin", F.array("r1", "r2", "r3")).withColumn("tri_ele_origin", F.array("r1_ele", "r2_ele", "r3_ele"))
df_tri_order = df_tri_order.withColumn("tri", sort_array("tri_origin", False)).drop("tri_origin") # sort_array("tri", False), False means descending order
df_tri_order = df_tri_order.withColumn("tri_ele", sort_array("tri_ele_origin", False)).drop("tri_ele_origin") # sort_array("tri", False), False means descending order

df_tri_order.printSchema()
df_tri_order.rdd.getNumPartitions()
#df_tri_order.show()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- tri_ele: array (nullable = false)
 |    |-- element: float (containsNull = true)



20

In [18]:
df_tri_order.show()

+---------+---+---+---+---------+---------+---------+---------------+--------------------+
|tri_order| r1| r2| r3|   r1_ele|   r2_ele|   r3_ele|            tri|             tri_ele|
+---------+---+---+---+---------+---------+---------+---------------+--------------------+
|      134|515|463|411|  1.08979| 0.707108| 0.382682|[515, 463, 411]|[1.08979, 0.70710...|
|      137|463|286|411| 0.707108|      0.0| 0.382682|[463, 411, 286]|[0.707108, 0.3826...|
|      239|126| 65| 78|-0.923883| -1.38269| -1.30657|  [126, 78, 65]|[-0.923883, -1.30...|
|      393|114| 80|125|     -1.0| -1.30657|-0.923884| [125, 114, 80]|[-0.923884, -1.0,...|
|      514|305|401|437|      0.0| 0.324426| 0.541198|[437, 401, 305]|[0.541198, 0.3244...|
|      515|365|305|437| 0.216774|      0.0| 0.541198|[437, 365, 305]|[0.541198, 0.2167...|
|      536|145|205|309|-0.765371|-0.382687|      0.0|[309, 205, 145]|[0.0, -0.382687, ...|
|      537|204|145|309|-0.382687|-0.765371|      0.0|[309, 204, 145]|[0.0, -0.382687, ...|

### get VT relation

In [17]:
# union and group vertices and get the preliminary VT relation
def grp_union(df_tri_order):
    '''
    df_tri_order: a DataFrame storing sorted extreme vertices of each triangle
    '''
    # groupby and collect_set
    df_tri_group1 = df_tri_order.groupBy('r1','r1_ele').agg(collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
    df_tri_group2 = df_tri_order.groupBy('r2','r2_ele').agg(collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
    df_tri_group3 = df_tri_order.groupBy('r3', 'r3_ele').agg(collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
    
    union12 = df_tri_group1.union(df_tri_group2) # the title of union12 will be the same as df_sj_group1
    union123 = union12.union(df_tri_group3) # the title of union123 will be the same as union12, so as df_sj_group1

    union123_group = union123.groupBy('r1','r1_ele').agg(collect_list('multi_tri').alias('multi_tri_list'), collect_list('multi_tri_ele').alias('multi_tri_ele_list'))
    
    # print("the schema of union123_group:", union123_group.printSchema())
    return union123_group

In [18]:
union123_group = grp_union(df_tri_order)
union123_group.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- multi_tri_list: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |-- multi_tri_ele_list: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: float (containsNull = true)



In [19]:
# sort the triangles in the preliminary VT relation
def get_multi_tri_order(tri_list, tri_ele_list):
    '''
    get_multi_tri_order is used to obtain the adjacent triangles, the results are in ascending order, but the triangle itself is in descending
    e.g., [[3, 2, 1], [5, 3, 2], [5, 3, 1]]
    tri_list: an array of array of array of integer
    tri_ele_list: an array of array of array of float
    '''

    tri = []
    tri_ele = []
    # pt_list.append(pt_self) # if we don't calculate roughness, we don't need self vertex
    for i in range(len(tri_list)):
        for j in range(len(tri_list[i])):
            # if tri_list[i][j] not in tri: # we do not need to check if it exists in tri_list, because we can prove that each one is unique
            tri.append(tri_list[i][j]) # update() will add multiple elements to a set, while add() only add one element to a set   
            tri_ele.append(tri_ele_list[i][j])
    
    tri_copy = copy.deepcopy(tri) # deep copy tri
    tri.sort() # sort the list of array, e.g., tri=[[3, 2, 1], [5, 2, 1], [5, 3, 1]], after sorting, [[3, 2, 1], [5, 2, 1], [5, 3, 1]]
    
    tri_ele_sort = []
    # sort tri_ele according to the same rule as tri
    for i in range(len(tri)):
        index_in_tri_origin = tri_copy.index(tri[i])
        tri_ele_sort.append(tri_ele[index_in_tri_origin])
    
    return tri, tri_ele_sort

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html

# StructType for get_multi_tri_order        
get_multi_tri_order_schema = StructType([
    StructField("multi_tri_sort", ArrayType(ArrayType(IntegerType())),True), 
    StructField('multi_tri_ele_sort',ArrayType(ArrayType(FloatType())),True)
])

get_multi_tri_order_udf = udf(get_multi_tri_order, get_multi_tri_order_schema)

df_VT = union123_group.withColumn("multi_tri_order", get_multi_tri_order_udf(union123_group.multi_tri_list, union123_group.multi_tri_ele_list))
df_VT = df_VT.select(col("r1").alias("Ver"), col("multi_tri_order.multi_tri_sort").alias("VT_filtra"), col("multi_tri_order.multi_tri_ele_sort").alias("VT_ele"))
df_VT.printSchema()
print("number of partitions for df_VT:", df_VT.rdd.getNumPartitions())

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VT_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)

number of partitions for df_VT: 20


### obtain the lower star of each vertex

In [20]:
# obtain Lower Star
def get_LS(Ver, VT_filtra, VT_ele):
# get_LS is used to obtain the edges and triangles of LS in ascending order
# VV: an array of VV relation in ascending order
    VV = set(num for sublist in VT_filtra for num in sublist if num != Ver)
    VV = sorted(VV)
        
    LS_edge = []
    for i in range(len(VV)):
        if VV[i] < int(Ver):
            LS_edge.append([int(Ver),VV[i]])
        
    LS_tri = []
    LS_tri_ele = []
    
    for i in range(len(VT_filtra)):
        if Ver >= max(VT_filtra[i]): # Ver_ele is not the maximum vertex
            LS_tri.append(VT_filtra[i])
            LS_tri_ele.append(VT_ele[i])
                
    return LS_edge, LS_tri, LS_tri_ele

# StructType for get_multi_tri_order        
get_LS_schema = StructType([
    StructField("LS_edge", ArrayType(ArrayType(IntegerType())),True), 
    StructField("LS_tri_filtra", ArrayType(ArrayType(IntegerType())),True),
    StructField('LS_tri_ele',ArrayType(ArrayType(FloatType())),True)
])

get_LS_udf = udf(get_LS, get_LS_schema)

df_LS = df_VT.withColumn("LS", get_LS_udf(df_VT.Ver, df_VT.VT_filtra, df_VT.VT_ele))
df_LS = df_LS.select("Ver", "VT_filtra", col("LS.LS_edge").alias("LS_edge"), col("LS.LS_tri_filtra").alias("LS_tri_filtra"), col("LS.LS_tri_ele").alias("LS_tri_ele"))
df_LS.printSchema()
print("number of partitions for df_LS:", df_LS.rdd.getNumPartitions())
# df_LS_edge.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_edge: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_tri_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_tri_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)

number of partitions for df_LS: 20


### compute Forman Gradient

In [21]:
from collections import deque

In [22]:
# get the Forman gradient from lower stars
def get_Forman(Ver, LS_edge, LS_filtra, LS_ele):
    '''
    get_Forman is used to compute Forman gradient
    LS_edge is triangles of lower star in ascending order
    LS_tri is triangles of lower star in ascending order
    '''
    # inner udf to get the number of unpaired edges for a triangle
    def num_unpaired_edges(tri_inner, crit_cell_inner, Forman_vec_pair_VE_inner, Forman_vec_pair_ET_inner):
        # compute the number of unpaired edges
        edge0 = [tri_inner[0], tri_inner[1]]
        edge1 = [tri_inner[0], tri_inner[2]]
        
        num_unpaired_edge = 2
        paired_edge = []
        for crit_inner_temp in crit_cell_inner:
            if len(crit_inner_temp) == 2: # crit_inner_temp is a critical edge
                paired_edge.append(crit_inner_temp)
                
        for pair_vec in Forman_vec_pair_VE_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, tri)
            paired_edge.append(pair_vec[1])
            # if type(pair_vec[0]) != int and len(pair_vec[0]) == 2: # pair_vec[0] is a paired edge, like in (edge, tri)
                # paired_edge.append(pair_vec[0])
            # if len(pair_vec[1]) == 2: # pair_vec[1] is a paired edge, like in (ver, edge)
                # paired_edge.append(pair_vec[1])
        for pair_vec in Forman_vec_pair_ET_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, tri)
            paired_edge.append(pair_vec[0])
                
        if edge0 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
        if edge1 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
            
        return num_unpaired_edge
    
    # inner udf to get the edge which is paired with a triangles
    def pair_an_edge_from_PQzero(alpha_inner, PQzero_inner):
    # alpha_inner is a triangle in [r0, r1, r2] format, and the vertext's elevation is in descending order
    # PQzero_inner is a queue storing all other edges
        alpha_e0 = [alpha_inner[0], alpha_inner[1]]
        alpha_e1 = [alpha_inner[0], alpha_inner[2]]
        for edge_inner in PQzero_inner:
            if edge_inner == alpha_e0:
                return edge_inner
            if edge_inner == alpha_e1:
                return edge_inner
            
    crit_cell = [] # crit_cell will store critical simplices
    # Forman_vec_pair = [] # Forman_vec_pair will store all Forman gradient pairs as a vector
    Forman_vec_pair_VE = []
    Forman_vec_pair_ET = []
    if (len(LS_edge)+len(LS_filtra)) == 0: # ver is a local minimum
        crit_cell.append([Ver]) # store minimum as an array [Ver] though it has only one element
    elif len(LS_edge) > 0:
        Forman_vec_pair_VE.append([Ver, LS_edge[0]]) # LS_edge[0] is the minimum edge since it is in ascending order
        
        # define two queues, PQzero storing all other edges, PQone storing all triangles which have only one unpaired edge
        PQzero = deque()
        PQone = deque()
        PQone_ele = deque()
        for i in range(1, len(LS_edge)): # add the other edges to PQzero
            PQzero.append(LS_edge[i])
            
        cell_temp_index_of_remove = []
        index_temp = 0
        for cell_temp in LS_filtra:
            if num_unpaired_edges(cell_temp, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                PQone.append(cell_temp) # storing cells with one unpaired edges, which are triangles in TIN
                PQone_ele.append(LS_ele[index_temp])
                # remove cell_temp from LS_filtra
                # LS_filtra.remove(cell_temp)
                cell_temp_index_of_remove.append(index_temp)
            index_temp += 1
            
        # remove cell_temp from LS_filtra and LS_ele
        for i in reversed(cell_temp_index_of_remove):
            del LS_filtra[i]
            del LS_ele[i]
                
        # sort PQone in ascending order, the initial queue is already sorted when creating
        # PQone = sorted(PQone) # sort() is not supported but we can use sorted()
        # PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
        while len(PQone) > 0 or len(PQzero) > 0:
            while len(PQone) > 0:
                # alpha = PQone.popleft() # alpha is a triangle in the PQone
                # alpha should be a triangle with lower elevation when PQone have more than one triangle
                if len(PQone) > 1 and abs(PQone_ele[0][1]-PQone_ele[1][1]) < 0.000001 and PQone_ele[0][2] > PQone_ele[1][2]:
                    alpha = PQone[1] # alpha is the second triangle
                    del PQone[1]
                    del PQone_ele[1]
                else:
                    alpha = PQone.popleft() # alpha is the first triangle, which is the same as alpha=PQone[0], then del PQone[0]
                    del PQone_ele[0]
                
                if num_unpaired_edges(alpha, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 0:
                    PQzero.append(alpha)
                else:
                    pair_alpha = pair_an_edge_from_PQzero(alpha, PQzero) # pair_alpha is an edge, [pair_alpha, alpha] will be a new paired vector
                    # if pair_alpha in PQzero:
                    PQzero.remove(pair_alpha) # remove pair_alpha from PQzero
                    # Forman_vec_pair.append([pair_alpha, alpha]) # add the [pair_alpha, alpha], which is [edge, tri]
                    Forman_vec_pair_ET.append([pair_alpha, alpha]) # add the [pair_alpha, alpha], which is [edge, tri]
                    
                    beta_cell_temp_index_of_remove = []
                    beta_index_temp = 0
                    for cell_beta in LS_filtra: # this can be optimized
                        if cell_beta not in PQone and num_unpaired_edges(cell_beta, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                            PQone.append(cell_beta)
                            PQone_ele.append(LS_ele[beta_index_temp])
                            # remove cell_beta from LS_tri
                            # LS_tri.remove(cell_beta)
                            beta_cell_temp_index_of_remove.append(beta_index_temp)
                        beta_index_temp += 1
                    
                    # remove cell_beta from LS_filtra and LS_ele
                    for i in reversed(beta_cell_temp_index_of_remove):
                        del LS_filtra[i]
                        del LS_ele[i]
                    
                    PQone = sorted(PQone)
                    PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
            if len(PQzero) > 0:
                gama = PQzero.popleft() # gama is a critical simplex
                crit_cell.append(gama)
                
                gama_cell_temp_index_of_remove = []
                gama_index_temp = 0
                for cell_gama in LS_filtra: # this can be optimized
                    if cell_gama not in PQone and num_unpaired_edges(cell_gama, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                        PQone.append(cell_gama)
                        PQone_ele.append(LS_ele[gama_index_temp])
                        # remove cell_gama from LS_tri
                        # LS_tri.remove(cell_gama)
                        gama_cell_temp_index_of_remove.append(gama_index_temp)
                    gama_index_temp += 1
                    
                # remove cell_beta from LS_filtra and LS_ele
                for i in reversed(gama_cell_temp_index_of_remove):
                    del LS_filtra[i]
                    del LS_ele[i]
                        
                PQone = sorted(PQone)
                PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
                
    return crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET

# StructType for Crit_cell        
add_Crit_cell_schema = StructType([
    StructField('Crit_cell',ArrayType(IntegerType()),True)
])

# StructType for VE pairs        
add_VE_schema = StructType([
    StructField("VE_pair_V", IntegerType(),True), 
    StructField('VE_pair_E',ArrayType(IntegerType()),True)
])

# StructType for ET pairs        
add_multi_ET_schema = StructType([
    StructField("ET_pair_E", ArrayType(IntegerType()),True), 
    StructField('ET_pair_T',ArrayType(IntegerType()),True)
])
 
# the whole StructType
get_Forman_schema = StructType([
    StructField("Crit_cell", ArrayType(ArrayType(IntegerType())), True),
    StructField("VE_pair", ArrayType(add_VE_schema), True),
    StructField("ET_pair", ArrayType(add_multi_ET_schema), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_Forman_udf = udf(get_Forman, get_Forman_schema)

In [23]:
df_Forman = df_LS.withColumn("LS_Forman", get_Forman_udf(df_LS.Ver, df_LS.LS_edge, df_LS.LS_tri_filtra, df_LS.LS_tri_ele))
df_Forman = df_Forman.select(col("Ver"), col("VT_filtra"), col("LS_Forman").alias("Forman"))
df_Forman.printSchema()
print("The number of partitions for df_Forman:", df_Forman.rdd.getNumPartitions())
# df_Forman.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Forman: struct (nullable = true)
 |    |-- Crit_cell: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- VE_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- ET_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- ET_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |    |    |-- ET_pair_T: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)

The number of par

In [24]:
# recording the time cost to compute the Forman gradient
t_df_Forman_0 = time.time()
num_df_Forman = df_Forman.count()
print("number of rows in df_Forman:", num_df_Forman)

t_df_Forman_1 = time.time()

time_Forman = t_df_Forman_1 - t_df_Forman_0
print("time for computing Forman gradient:", time_Forman)

number of rows in df_Forman: 625
time for computing Forman gradient: 29.716614484786987


##### save df_Forman to HDFS

In [42]:
# df_Forman = df_Forman.repartition(20)
t_save_0 = time.time()

# file_df_Forman = directory + '/' + tin_filename + '_sortForman' + '.parquet'
# df_Forman.write.parquet(file_df_Forman) 

t_save_1 = time.time()
print("Time cost to save df_Forman in Parquet:", t_save_1 - t_save_0)

Time cost to save df_Forman in Parquet: 2.4177093505859375


# Extract critical vertices, edeges, and triangles

In [24]:
df_crit = df_Forman.select("Ver", "VT_filtra", col("Forman.Crit_cell").alias("Critical"))
print("number of partitions for df_crit:", df_crit.rdd.getNumPartitions())
df_crit.printSchema()
# df_crit.show()

number of partitions for df_crit: 20
root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [25]:
# split the Critical column to obtain the critical vertices, edges, and triangles
def get_crit_VET(crit_cell):
    if len(crit_cell) < 1: # crit_cell is empty, there are no critical simplices within this lower star
        return
    else:
        # initialize the critical vertices, critical edges, and critical triangles
        crit_ver = None
        crit_edge = []
        crit_tri = []
        for icrit_cell_temp in crit_cell:
            if len(icrit_cell_temp) == 1: # icrit_cell_temp stores a critical vertex
                crit_ver = icrit_cell_temp[0]
            if len(icrit_cell_temp) == 2: # icrit_cell_temp stores critical edges
                crit_edge.append(icrit_cell_temp)
            if len(icrit_cell_temp) == 3: # icrit_cell_temp stores critical triangles
                crit_tri.append(icrit_cell_temp)
        if len(crit_edge) == 0:
            crit_edge = None
        if len(crit_tri) == 0:
            crit_tri = None
        return crit_ver, crit_edge, crit_tri
        
# the whole StructType
get_crit_VET_schema = StructType([
    StructField("crit_ver", IntegerType(), True),
    StructField("crit_edge", ArrayType(ArrayType(IntegerType())), True),
    StructField("crit_tri", ArrayType(ArrayType(IntegerType())), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_crit_VET_udf = udf(get_crit_VET, get_crit_VET_schema)

df_crit_VET = df_crit.withColumn("crit_cell", get_crit_VET_udf(df_crit.Critical))
print("number of partitions for df_crit_VET:", df_crit_VET.rdd.getNumPartitions())
df_crit_VET.printSchema()
# df_crit_VET.show()

number of partitions for df_crit_VET: 20
root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- crit_cell: struct (nullable = true)
 |    |-- crit_ver: integer (nullable = true)
 |    |-- crit_edge: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- crit_tri: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)



In [26]:
# to get critical vertices
df_crit_V = df_crit_VET.select(col("Ver"), col("crit_cell.crit_ver").alias("Min_ver")).na.drop()
df_crit_V.printSchema()
print("number of partitions for df_crit_V:", df_crit_V.rdd.getNumPartitions())
# df_crit_V.show()

root
 |-- Ver: integer (nullable = true)
 |-- Min_ver: integer (nullable = true)

number of partitions for df_crit_V: 20


In [27]:
num_crit_V = df_crit_V.count()
print("number of critical vertices:", num_crit_V)

number of critical vertices: 5


In [28]:
# to get critical edges
df_crit_E_init = df_crit_VET.select(col("Ver"), col("crit_cell.crit_edge").alias("Saddle_edge_init")).na.drop()

# there could be multiple saddles within one lower star
df_crit_E = df_crit_E_init.select(col("Ver"), explode("Saddle_edge_init").alias("Saddle_edge"))
df_crit_E.printSchema()
print("number of partitions for df_crit_E:", df_crit_E.rdd.getNumPartitions())
# df_crit_E.show()

root
 |-- Ver: integer (nullable = true)
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_E: 20


In [29]:
num_crit_E = df_crit_E.count()
print("number of critical edges:", num_crit_E)

number of critical edges: 5


In [30]:
# to get critical triangles
df_crit_T_init = df_crit_VET.select(col("Ver"), col("crit_cell.crit_tri").alias("Max_tri_init")).na.drop()

# there could be multiple critical triangles within one lower star
df_crit_T = df_crit_T_init.select(col("Ver"), explode("Max_tri_init").alias("Max_tri")).na.drop()
df_crit_T.printSchema()
print("number of partitions for df_crit_T:", df_crit_T.rdd.getNumPartitions())
# df_crit_T.show()

root
 |-- Ver: integer (nullable = true)
 |-- Max_tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_T: 20


In [31]:
num_crit_T = df_crit_T.count()
print("number of critical triangles:", num_crit_T)

number of critical triangles: 1


# Methods to build graph V1

In [29]:
# to obtain critical simplices and VE_paris from the Forman gradient
df_Forman_VE_pairs = df_Forman.select(col("Ver"), col("Forman.Crit_cell").alias("Crit_cell"), col("Forman.VE_pair").alias("VE_pair"))
df_Forman_VE_pairs.printSchema()
# df_Forman_VE_pairs.show()

root
 |-- Ver: integer (nullable = true)
 |-- Crit_cell: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VE_pair: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [43]:
# df_Forman_VE_pairs = df_Forman_VE_pairs.repartition(20)
t_save_0 = time.time()

# file_df_Forman_VE_pairs = directory + '/' + tin_filename + '_VE_Pairs' + '.parquet'
# df_Forman_VE_pairs.write.format("parquet").mode("overwrite").save(file_df_Forman_VE_pairs)

t_save_1 = time.time()
print("Time cost to save df_Forman_VE_pairs in Parquet:", t_save_1 - t_save_0)

Time cost to save df_Forman_VE_pairs in Parquet: 2.0288069248199463


In [30]:
# to get the DataFrame containg all vertices and VE pairs 
df_graph_V1 = df_Forman_VE_pairs.select(col("Ver").alias("V1_node"), df_Forman_VE_pairs.VE_pair.VE_pair_E.alias("V1_edge"))
df_graph_V1.printSchema()

root
 |-- V1_node: integer (nullable = true)
 |-- V1_edge: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



### nodes of a graph

In [31]:
df_graph_V1_node = df_graph_V1.select(col("V1_node").alias("id"))
df_graph_V1_node.printSchema()
# df_graph_V1_node.show()

root
 |-- id: integer (nullable = true)



##### repartition and save df_graph_V1_node

In [39]:
# df_graph_V1_node = df_graph_V1_node.repartition(20)
t_save_node_0 = time.time()

# file_df_graph_V1_node = 'SigSpatial_data' + '/' + tin_filename + '_VE_node_LPA' + '.parquet'
# df_graph_V1_node_join.write.parquet(file_df_graph_V1_node_join) 

t_save_node_1 = time.time()
print("Time cost to save Graph_VE nodes in Parquet:", t_save_node_1 - t_save_node_0)

Time cost to save Graph_VE nodes in Parquet: 2.9325485229492188e-05


### arcs of a graph

In [32]:
df_graph_V1_arc = df_graph_V1.select(df_graph_V1.V1_edge[0][0].alias('src'), df_graph_V1.V1_edge[0][1].alias('dst')).na.drop()
df_graph_V1_arc.printSchema()
print("number of partitions for df_graph_V1_arc:", df_graph_V1_arc.rdd.getNumPartitions())
# df_graph_V1_arc.show(30)

root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)

number of partitions for df_graph_V1_arc: 4


##### repartition and save the dataframe df_graph_V1_arc

In [41]:
# df_graph_V1_arc = df_graph_V1_arc.repartition(20)
# print("number of partitions for df_graph_V1_arc:", df_graph_V1_arc.rdd.getNumPartitions())

# save Graph_VE arcs to Parquet
t_save_arc_0 = time.time()
# file_df_graph_V1_arc = 'SigSpatial_data' + '/' + tin_filename + '_VE_arc_LPA' + '.parquet'
# df_graph_V1_arc.write.parquet(file_df_graph_V1_arc) 

t_save_arc_1 = time.time()
print("Time cost to save Graph_VE arcs in Parquet:", t_save_arc_1 - t_save_arc_0)

Time cost to save Graph_VE arcs in Parquet: 3.0040740966796875e-05


### construct a Graph_VE

In [33]:
graph_VE = GraphFrame(df_graph_V1_node, df_graph_V1_arc)

In [34]:
# set a checkpoint directory to improve performance
# Checkpointing regularly helps recover from failures, clean shuffle files, shorten the
# lineage of the computation graph, and reduce the complexity of plan optimization.

spark.sparkContext.setCheckpointDir('checkpoints')

##### extract connected components

In [35]:
t_con_0 = time.time()
# result_con consists of two columns, vertex id, component
result_con_VE = graph_VE.connectedComponents()
result_con_VE.printSchema()

t_con_1 = time.time()
t_con = t_con_1 - t_con_0
print("time cost of connected components:",t_con)

root
 |-- id: integer (nullable = true)
 |-- component: long (nullable = true)

time cost of connected components: 23.585004091262817


##### 1) get the boundary vertices of saddles

In [36]:
df_crit_E_v0 = df_crit_E.select(col("Saddle_edge"), df_crit_E.Saddle_edge[0].alias("Saddle_pt0"))
df_crit_E_v0.printSchema()

root
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Saddle_pt0: integer (nullable = true)



In [37]:
df_crit_E_v1 = df_crit_E.select(col("Saddle_edge"), df_crit_E.Saddle_edge[1].alias("Saddle_pt1"))
df_crit_E_pts = df_crit_E_v0.union(df_crit_E_v1).withColumnRenamed("Saddle_pt0", "Saddle_bound_pt")
df_crit_E_pts.printSchema()

root
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Saddle_bound_pt: integer (nullable = true)



##### 2) inner join result_con with the boundary vertices of saddles to identify the saddles that belong to the same component; typically a saddle belongs to 2 component in Graph_VE

In [38]:
# broadcast left join connected components with saddles
result_con_saddle_VE = result_con_VE.join(df_crit_E_pts,result_con_VE.id==df_crit_E_pts.Saddle_bound_pt, "inner")
result_con_saddle_VE.printSchema()

root
 |-- id: integer (nullable = true)
 |-- component: long (nullable = true)
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Saddle_bound_pt: integer (nullable = true)



In [39]:
saddles_per_con_VE = result_con_saddle_VE.groupBy('component').agg(collect_list('Saddle_edge').alias('multi_Saddles'))
saddles_per_con_VE.printSchema()

root
 |-- component: long (nullable = true)
 |-- multi_Saddles: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [63]:
# saddles_per_con_VE = saddles_per_con_VE.repartition(20)
t_save_0 = time.time()

file_saddles_per_con_VE = directory + '/' + tin_filename + '_saddles_per_con_VE' + '.parquet'
saddles_per_con_VE.write.parquet(file_saddles_per_con_VE) 

t_save_1 = time.time()
print("Time cost to save saddles_per_con_VE in Parquet:", t_save_1 - t_save_0)

Time cost to save saddles_per_con_VE in Parquet: 9.31522536277771


In [ ]:
# get the potential saddle to be simplified within each component
# the potential saddle-minimum pair should have the minimal persistence value among all saddle-minimum pairs
def get_mini_saddle_VE(component, multi_Saddles):
    # get_LS is used to obtain the edges and triangles of LS in ascending order
    # component: the id or index of the minimum of this connected component in Graph_VE
    if len(multi_Saddles) > 0:
        # initialize the potential minimal saddle
        mini_saddle = multi_Saddles[0]
        mini_saddle_persist = mini_saddle[0] # mini_saddle is a pair of vertex, and mini_saddle[0] > mini_saddle[1]
        
        for i in range(1, len(multi_Saddles)):
            if multi_Saddles[i][0] < mini_saddle_persist: # multi_Saddles[i][0] is the persist value of multi_Saddles[i]
                mini_saddle = multi_Saddles[i]
                mini_saddle_persist = mini_saddle[0]                
                
        return mini_saddle
    else:
        return None

get_mini_saddle_VE_udf = udf(get_mini_saddle_VE, ArrayType(IntegerType()))

saddles_per_con_VE_mini = saddles_per_con_VE.withColumn(get_mini_saddle_VE_udf(saddles_per_con_VE.component, saddles_per_con_VE.multi_Saddles))
saddles_per_con_VE_mini.printSchema()

# Methods to build Graph_ET

### prepare nodes of Graph ET

In [32]:
# all triangles are nodes of graph V2
df_graph_V2_node = df_tri_order.select(col("tri_order").alias("id"), col("tri"))
df_graph_V2_node.printSchema()

root
 |-- id: integer (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [61]:
# add a special triangle [-1, -1, -1] as node -1; this is useful to deal with boundary elements
# e.g., for a boundary edge, it has only one coboundary triangle; to guarantee a consistent encoding, each edge has two coboundary triangels, the other coboundary triangle for a boundary edge is [-1,-1,-1]
# columns_special = ['tri', 'V2_node']
value_special = [(-1, [-1, -1, -1])]
schema_special = StructType([
    StructField('tri_order', IntegerType(),True),
    StructField("tri", ArrayType(IntegerType()),True)    
])
df_special_V2_node = spark.createDataFrame(value_special, schema_special)
df_special_V2_node.printSchema()

root
 |-- tri_order: integer (nullable = true)
 |-- tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)



##### nodes of Graph_ET

In [62]:
df_graph_V2_node = df_graph_V2_node.union(df_special_V2_node)
df_graph_V2_node_final = df_graph_V2_node.select(col("tri_order").alias("id"))
df_graph_V2_node_final.printSchema()

root
 |-- id: integer (nullable = true)



### prepare arcs of Graph_ET

In [33]:
# Method: firstly extract df_Forman_ET_pair from df_Forman
df_Forman_ET_pair = df_Forman.select(col("Ver"), col("VT_filtra"), col("Forman.ET_pair").alias("ET_pair"))
df_Forman_ET_pair.printSchema()
# df_Forman_ET_pair.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- ET_pair: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ET_pair_E: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- ET_pair_T: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [34]:
# retrieve co-boundary triangles (ET relation)
def get_co_tri(VT, ET_pair):
    if len(ET_pair) > 0:
        co_tri_pair = []
        num_ET_pair = len(ET_pair) # obtain the number of ET_pairs
        def Co_tri(e,t,vt):
            # e is sorted
            for t_temp in vt:
                # e0 = sorted([vt[0], vt[1]], reverse=True) # sort the edge in descending order
                e0 = [t_temp[0], t_temp[1]] # e0, e1, and e2 are in descending order by default
                e1 = [t_temp[0], t_temp[2]]
                e2 = [t_temp[1], t_temp[2]]
                t = [int(t[0]), int(t[1]), int(t[2])]
                t_temp = [int(t_temp[0]), int(t_temp[1]), int(t_temp[2])]
                # if e==e0 or e==e1 or e==e2 and t != t_temp:
                if e==e0 or e==e1 or e==e2:
                    if t != t_temp:
                        return t_temp # t_temp is a coboundary triangle of t in vt
                
        for i in range(num_ET_pair):
            ET_pair_e = ET_pair[i]['ET_pair_E'] # the i-th ET_pair
            ET_pair_t = ET_pair[i]['ET_pair_T']
            ET_pair_co_t = Co_tri(ET_pair_e, ET_pair_t, VT)
            # if ET_pair_co_t is None: # len(ET_pair_co_t) is empty, boundary triangles
                # ET_pair_co_t = [-1, -1, -1]
            co_tri_pair.append([ET_pair_co_t, ET_pair_t])
            
        return co_tri_pair
    
# the whole StructType
get_co_tri_schema = StructType([
    StructField("ET_pair_co_t", ArrayType(IntegerType()), True),
    StructField("ET_pair_t", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_co_tri_udf = udf(get_co_tri, ArrayType(get_co_tri_schema))

df_Forman_V2_edge = df_Forman_ET_pair.withColumn("ET_pair_inTri", get_co_tri_udf(df_Forman_ET_pair.VT_filtra, df_Forman_ET_pair.ET_pair))
df_Forman_V2_edge = df_Forman_V2_edge.select(col("Ver"), col("ET_pair_inTri"))
df_Forman_V2_edge.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- ET_pair_inTri: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ET_pair_co_t: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- ET_pair_t: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



##### construct Graph_ET reversely, set src nodes as dst nodes, and dst nodes as src nodes

In [36]:
# explode df_Forman_V2_edge
df_Forman_V2_edge_split = df_Forman_V2_edge.select(explode(df_Forman_V2_edge.ET_pair_inTri).alias("ET_pair_src_dst"))

df_Forman_V2_edge_split = df_Forman_V2_edge_split.select(col("ET_pair_src_dst.ET_pair_co_t").alias("arc_dst"), col("ET_pair_src_dst.ET_pair_t").alias("arc_src")).na.drop()
df_Forman_V2_edge_split.printSchema()

root
 |-- arc_dst: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- arc_src: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [37]:
# join two DataFrames by df_Forman_V2_edge_split.arc_src == df_Forman_V2_edge_split.arc_des
df_Forman_V2_edge_split_src = df_Forman_V2_edge_split.join(df_graph_V2_node, df_Forman_V2_edge_split.arc_src == df_graph_V2_node.tri, "left")
df_Forman_V2_edge_split_src = df_Forman_V2_edge_split_src.select(col("arc_dst"), col("id").alias("src"))
df_Forman_V2_edge_split_src.printSchema()

root
 |-- arc_dst: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- src: integer (nullable = true)



In [38]:
# join two DataFrames by df_Forman_V2_edge_split.arc_src == df_Forman_V2_edge_split.arc_des
df_Forman_V2_edge_split_dst = df_Forman_V2_edge_split_src.join(df_graph_V2_node, df_Forman_V2_edge_split_src.arc_dst == df_graph_V2_node.tri, "left")
df_Forman_V2_edge_split_dst = df_Forman_V2_edge_split_dst.select(col("src"), col("id").alias("dst"))

##### arcs of Graph_ET

In [39]:
df_graph_V2_edge_final = df_Forman_V2_edge_split_dst
df_graph_V2_edge_final.printSchema()

root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)



### Construct a Graph_ET

In [40]:
graph_ET = GraphFrame(df_graph_V2_node, df_graph_V2_edge_final)

print("type of graph_V2:", type(graph_ET))

type of graph_V2: <class 'graphframes.graphframe.GraphFrame'>


In [41]:
# set a checkpoint directory to improve performance
# Checkpointing regularly helps recover from failures, clean shuffle files, shorten the
# lineage of the computation graph, and reduce the complexity of plan optimization.

spark.sparkContext.setCheckpointDir('checkpoints')

##### extract connected components

In [42]:
t_con_0 = time.time()
# result_con consists of two columns, vertex id, component
result_con_ET = graph_ET.connectedComponents()
result_con_ET.printSchema()

t_con_1 = time.time()
t_con = t_con_1 - t_con_0
print("time cost of connected components:",t_con)

root
 |-- id: integer (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- component: long (nullable = true)

time cost of connected components: 28.667405128479004


##### 1)  get the co-boundary triangles of saddles

In [90]:
# to get critical edges
df_crit_E_VT = df_crit_VET.select(col("Ver"), col("VT_filtra"), col("crit_cell.crit_edge").alias("Saddle_edge_init")).na.drop()

# there could be multiple saddles within one lower star
df_crit_E_VT = df_crit_E_VT.select(col("Ver"), col("VT_filtra"), explode("Saddle_edge_init").alias("Saddle_edge"))
df_crit_E_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [91]:
# retrieve co-boundary triangles
def get_saddle_edge_tri(Saddle_edge, VT):
    if len(VT) > 0:
        e = Saddle_edge
        tri_12 = []
        num_VT = len(VT) # obtain the number of ET_pairs
        for t_temp in VT:
            e0 = [t_temp[0], t_temp[1]] # e0, e1, and e2 are in descending order by default
            e1 = [t_temp[0], t_temp[2]]
            e2 = [t_temp[1], t_temp[2]]
            if e==e0 or e==e1 or e==e2:
                tri_12.append(t_temp)
            # if len(tri_12) == 2: # Maybe this triangle is at the boundary, so it has only one corresponding triangle in VT
                # break
        '''
        if len(tri_12) == 1:
            t_special = [-1,-1,-1]
            tri_12.append(t_special)
        '''
            
        return tri_12
    
# the whole StructType
get_saddle_edge_tri_schema = StructType([
    StructField("Saddle_co_t1", ArrayType(IntegerType()), True),
    StructField("Saddle_co_t2", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_saddle_edge_tri_udf = udf(get_saddle_edge_tri, ArrayType(ArrayType(IntegerType())))

df_crit_E_tri = df_crit_E_VT.withColumn("saddle_edge_Tri", get_saddle_edge_tri_udf(df_crit_E_VT.Saddle_edge, df_crit_E_VT.VT_filtra))
df_crit_E_tri = df_crit_E_tri.select("Saddle_edge", "saddle_edge_Tri")
df_crit_E_tri.printSchema()
# df_crit_E_tri.show()

root
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- saddle_edge_Tri: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [92]:
df_crit_E_Co_t0 = df_crit_E_tri.select("Saddle_edge", df_crit_E_tri.saddle_edge_Tri[0].alias("Saddle_Co_t0"))
df_crit_E_Co_t1 = df_crit_E_tri.select("Saddle_edge", df_crit_E_tri.saddle_edge_Tri[1].alias("Saddle_Co_t1"))

df_crit_E_Co_tris = df_crit_E_Co_t0.union(df_crit_E_Co_t1)
df_crit_E_Co_tris = df_crit_E_Co_tris.withColumnRenamed("Saddle_Co_t0", "Saddle_Co_t")

df_crit_E_Co_tris.printSchema()

root
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Saddle_Co_t: array (nullable = true)
 |    |-- element: integer (containsNull = true)



##### 2) inner join result_con with the co-boundary triangles of saddles to identify the saddles that belong to the same component; typically a saddle belongs to 2 component in Graph_ET

In [67]:
# broadcast left join connected components with saddles
result_con_saddle_ET = result_con_ET.join(df_crit_E_Co_tris,result_con_ET.tri==df_crit_E_Co_tris.Saddle_Co_t, "inner")
result_con_saddle_ET.printSchema()

root
 |-- id: integer (nullable = true)
 |-- tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- component: long (nullable = true)
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Saddle_Co_t: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [68]:
saddles_per_con_ET = result_con_saddle_ET.groupBy('component').agg(collect_list('Saddle_edge').alias('multi_Saddles'))
saddles_per_con_ET.printSchema()

root
 |-- component: long (nullable = true)
 |-- multi_Saddles: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



##### 3) inner join result_con with critical triangles to identify the component id of critical triangle 

In [75]:
df_crit_T_con = df_crit_T.join(result_con_ET, df_crit_T.Max_tri==result_con_ET.tri)
df_crit_T_con.printSchema()

root
 |-- Max_tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- id: integer (nullable = true)
 |-- tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- component: long (nullable = true)



In [47]:
# join two DataFrames to obatin VE relation
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_crit_E_VT = df_crit_E.join(df_VT, df_crit_E.Ver == df_VT.Ver, "left")
df_crit_E_VT = df_crit_E_VT.select(df_crit_E.Ver, "Saddle_edge", "VT_filtra")
df_crit_E_VT.printSchema()
# df_crit_E_VT.show()

root
 |-- Ver: integer (nullable = true)
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [48]:
# retrieve co-boundary triangles
def get_saddle_edge_tri(Saddle_edge, VT):
    if len(VT) > 0:
        e = Saddle_edge
        tri_12 = []
        num_VT = len(VT) # obtain the number of ET_pairs
        for t_temp in VT:
            e0 = [t_temp[0], t_temp[1]] # e0, e1, and e2 are in descending order by default
            e1 = [t_temp[0], t_temp[2]]
            e2 = [t_temp[1], t_temp[2]]
            if e==e0 or e==e1 or e==e2:
                tri_12.append(t_temp)
            # if len(tri_12) == 2: # Maybe this triangle is at the boundary, so it has only one corresponding triangle in VT
                # break
        '''
        if len(tri_12) == 1:
            t_special = [-1,-1,-1]
            tri_12.append(t_special)
        '''
            
        return tri_12
    
# the whole StructType
get_saddle_edge_tri_schema = StructType([
    StructField("Saddle_edge_t1", ArrayType(IntegerType()), True),
    StructField("Saddle_edge_t2", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_saddle_edge_tri_udf = udf(get_saddle_edge_tri, ArrayType(ArrayType(IntegerType())))

df_crit_E_tri = df_crit_E_VT.withColumn("saddle_edge_Tri", get_saddle_edge_tri_udf(df_crit_E_VT.Saddle_edge, df_crit_E_VT.VT_filtra))
df_crit_E_tri = df_crit_E_tri.select(col("saddle_edge_Tri"))
df_crit_E_tri.printSchema()
# df_crit_E_tri.show()

root
 |-- saddle_edge_Tri: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [56]:
# obtain the saddle tri
df_crit_E2tri = df_crit_E_tri.select(explode("saddle_edge_Tri").alias("saddle_edge_Tri_split"))
# some nodes may belong to multiple saddles, we need to remove them
df_crit_E2tri = df_crit_E2tri.distinct()
df_crit_E2tri.printSchema()

root
 |-- saddle_edge_Tri_split: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [58]:
# left join df_crit_E2tri with df_graph_V2_node to obtain corresponding id of tri
df_crit_E2tri = df_crit_E2tri.join(df_graph_V2_node, df_crit_E2tri.saddle_edge_Tri_split == df_graph_V2_node.tri, "left")
df_crit_E2tri = df_crit_E2tri.select(col('id').alias('saddle_edge_Tri_split_id'))
df_crit_E2tri.printSchema()

root
 |-- saddle_edge_Tri_split_id: integer (nullable = true)



In [60]:
# identify the saddle tri that each component have
result_con_saddle = result_con_ET.join(df_crit_E2tri,result_con_ET.id==df_crit_E2tri.saddle_edge_Tri_split_id, "inner")

print("number of partitions for result_con_saddle:", result_con_saddle.rdd.getNumPartitions())
result_con_saddle.printSchema()
# result_con_saddle.show(30)

number of partitions for result_con_saddle: 20
root
 |-- id: integer (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- component: long (nullable = true)
 |-- saddle_edge_Tri_split_id: integer (nullable = true)



In [61]:
# obtain all saddle tris for each connected component
unique_SdlTris_per_con = result_con_saddle.groupBy('component').agg(collect_list('id').alias('multi_SdlTris'))
unique_SdlTris_per_con.printSchema()

root
 |-- component: long (nullable = true)
 |-- multi_SdlTris: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [62]:
# left join df_graph_V1_arc with result_con to identify which component that each arc belongs to
df_G_V2_arc_component = df_graph_V2_edge_final.join(result_con_ET,df_graph_V2_edge_final.src==result_con_ET.id, "left").drop('id')
df_G_V2_arc_component.printSchema()

root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)
 |-- tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- component: long (nullable = true)



In [63]:
from pyspark.sql.functions import create_map

In [64]:
# convert each arc to a dictionary
df_G_V2_arc_component_dict = df_G_V2_arc_component.withColumn("arcs", create_map(col('src'), col('dst'))).drop('src','dst')

# combine dictionaries belonging to an identical component to one list
df_G_V2_arc_component_dict_gpy = df_G_V2_arc_component_dict.groupBy('component').agg(collect_list('arcs').alias('subgraphs'))

# left join df_G_V1_arc_component_dict_gpy with unique_SdlPts_per_con to obtain SdlPts belonging to each component
df_G_V2_arc_component_dict_gpy = df_G_V2_arc_component_dict_gpy.join(unique_SdlTris_per_con,df_G_V2_arc_component_dict_gpy.component==unique_SdlTris_per_con.component, "left").select(df_G_V2_arc_component_dict_gpy.component, 'multi_SdlTris', 'subgraphs')
df_G_V2_arc_component_dict_gpy.printSchema()

root
 |-- component: long (nullable = true)
 |-- multi_SdlTris: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- subgraphs: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: integer
 |    |    |-- value: integer (valueContainsNull = true)



In [65]:
def bfs_rdd(component, multi_SdlTris, subgraphs):
    # Perform a breadth-first search on a single graph
    
    # subgraphs is a list of dictionaries, we need to convert it to a single dictionary
    subgraphs_dict = {}
    # merge all sub-dictionaries to one single dictionary
    for d in subgraphs:
        subgraphs_dict = {**subgraphs_dict, **d}
        
    visited = set()
    queue = multi_SdlTris
    while queue:
        node = queue.pop(0)
        print("node:",node)
        if node not in visited:
            visited.add(node)
            neighbors = subgraphs_dict.get(node, [])
            if neighbors != []:
                queue.append(neighbors)
                
    visited = list(visited)
    return component, visited

In [66]:
# Use mapPartitions to apply the bfs function to each partition of the RDD
# Each partition corresponds to a single graph that is processed by a single executor
t_V2_0 = time.time()

df_V2_result = df_G_V2_arc_component_dict_gpy.rdd.map(lambda partition: bfs_rdd(partition['component'], partition['multi_SdlTris'], partition['subgraphs'])).toDF()
# V2_result_col = V2_result.collect()
num_df_V2_result = df_V2_result.count()
print("num_df_V2_result:", num_df_V2_result)

t_V2_1 = time.time()
print("Time cost to extract V2-Paths:", t_V2_1 - t_V2_0)

num_df_V2_result: 29
Time cost to extract V2-Paths: 9.146892070770264


In [67]:
df_V2_result_col = df_V2_result.collect()

In [68]:
len(df_V2_result_col)

29

In [71]:
df_V2_result_col[0:10]

[Row(_1=4, _2=[]),
 Row(_1=817, _2=[]),
 Row(_1=1009, _2=[]),
 Row(_1=388, _2=[]),
 Row(_1=578, _2=[]),
 Row(_1=384, _2=[]),
 Row(_1=0, _2=[]),
 Row(_1=6, _2=[]),
 Row(_1=40, _2=[]),
 Row(_1=400, _2=[512, 513, 896, 897, 654, 1040, 785, 1041, 787, 786, 789, 657, 792, 793, 794, 795, 796, 797, 414, 798, 799, 417, 802, 801, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 560, 561, 944, 945, 1088, 1089, 704, 705, 462, 463, 846, 849, 606, 736, 993, 992, 739, 740, 609, 742, 743, 745, 750, 1136, 1137, 752, 755, 753, 510])]

In [72]:
df_V2_result_col[0]['_2']

[]

In [87]:
valid_v2 = 0
for i in range(len(df_V2_result_col)):
    if len(df_V2_result_col[i]['_2']) > 0:
        valid_v2 += 1
        print(df_V2_result_col[i])
        print(len(df_V2_result_col[i][1]))        
        print("****************************")

Row(_1=400, _2=[512, 513, 896, 897, 654, 1040, 785, 1041, 787, 786, 789, 657, 792, 793, 794, 795, 796, 797, 414, 798, 799, 417, 802, 801, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 560, 561, 944, 945, 1088, 1089, 704, 705, 462, 463, 846, 849, 606, 736, 993, 992, 739, 740, 609, 742, 743, 745, 750, 1136, 1137, 752, 755, 753, 510])
65
****************************
Row(_1=30, _2=[224, 321, 225, 128, 129, 32, 33, 270, 271, 368, 369, 176, 177, 81, 80, None, 222, 318])
18
****************************
Row(_1=398, _2=[768, 769, 770, 771, 772, 773, 774, 775, 776, 778, 779, 780, 781, 782, 783, 726, None, 729])
18
****************************
Row(_1=14, _2=[14, 16, 17, 18, 19, 21, None])
7
****************************
Row(_1=10, _2=[10, 12, 13, 15, None])
5
****************************


In [74]:
valid_v2

5

In [76]:
df_crit_T.show()

+---+---------------+
|Ver|        Max_tri|
+---+---------------+
|624|[624, 620, 607]|
+---+---------------+



In [86]:
len(df_V2_result_col[1][1])

0

In [88]:
18+18+7+5

48

In [93]:
df_crit_E_Co_tris_col = df_crit_E_Co_tris.collect()

In [94]:
df_crit_E_Co_tris_col

[Row(Saddle_edge=[322, 274], Saddle_Co_t=[325, 322, 274]),
 Row(Saddle_edge=[299, 272], Saddle_Co_t=[350, 299, 272]),
 Row(Saddle_edge=[278, 277], Saddle_Co_t=[279, 278, 277]),
 Row(Saddle_edge=[321, 273], Saddle_Co_t=[351, 321, 273]),
 Row(Saddle_edge=[345, 275], Saddle_Co_t=[355, 345, 275]),
 Row(Saddle_edge=[322, 274], Saddle_Co_t=None),
 Row(Saddle_edge=[299, 272], Saddle_Co_t=[358, 299, 272]),
 Row(Saddle_edge=[278, 277], Saddle_Co_t=[356, 278, 277]),
 Row(Saddle_edge=[321, 273], Saddle_Co_t=[359, 321, 273]),
 Row(Saddle_edge=[345, 275], Saddle_Co_t=None)]

In [96]:
for i in range(len(df_tri_order_col)):
    if df_tri_order_col[i]['tri'] == [325, 322, 274] or df_tri_order_col[i]['tri'] == [350, 299, 272] or df_tri_order_col[i]['tri'] == [279, 278, 277] or df_tri_order_col[i]['tri'] == [351, 321, 273] or df_tri_order_col[i]['tri'] == [355, 345, 275] or df_tri_order_col[i]['tri'] == [358, 299, 272] or df_tri_order_col[i]['tri'] == [356, 278, 277] or df_tri_order_col[i]['tri'] == [359, 321, 273]:
        print(df_tri_order_col[i])

Row(tri_order=417, r1=299, r2=272, r3=350, r1_ele=0.0, r2_ele=-0.07611879706382751, r3_ele=0.0761198028922081, tri=[350, 299, 272], tri_ele=[0.0761198028922081, 0.0, -0.07611879706382751])
Row(tri_order=1137, r1=355, r2=275, r3=345, r1_ele=0.07612019777297974, r2_ele=-0.07611879706382751, r3_ele=0.0, tri=[355, 345, 275], tri_ele=[0.07612019777297974, 0.0, -0.07611879706382751])
Row(tri_order=369, r1=358, r2=272, r3=299, r1_ele=0.07612189650535583, r2_ele=-0.07611879706382751, r3_ele=0.0, tri=[358, 299, 272], tri_ele=[0.07612189650535583, 0.0, -0.07611879706382751])
Row(tri_order=15, r1=356, r2=277, r3=278, r1_ele=0.07612189650535583, r2_ele=0.0, r3_ele=0.0, tri=[356, 278, 277], tri_ele=[0.07612189650535583, 0.0, 0.0])
Row(tri_order=14, r1=277, r2=279, r3=278, r1_ele=0.0, r2_ele=0.0, r3_ele=0.0, tri=[279, 278, 277], tri_ele=[0.0, 0.0, 0.0])
Row(tri_order=785, r1=321, r2=351, r3=273, r1_ele=0.0, r2_ele=0.0761198028922081, r3_ele=-0.07611879706382751, tri=[351, 321, 273], tri_ele=[0.07611

In [95]:
df_tri_order.printSchema()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- tri_ele: array (nullable = false)
 |    |-- element: float (containsNull = true)



In [97]:
sassles_co_tri_order_list = [417,1137,369,15,14,785,782,814]

In [99]:
for i in range(len(df_V2_result_col)):
    if df_V2_result_col[i]['_1'] == 400:
        for temp in sassles_co_tri_order_list:
            if temp in df_V2_result_col[i]['_2']:
                print(temp)

417
1137
785
814
